## Gemini

In [5]:
import os
from dotenv import load_dotenv
from google import genai
import json

# 加载环境变量
load_dotenv()

# 获取配置
api_key = os.getenv("GEMINI_API_KEY")
model_name = os.getenv("GEMINI_MODEL")
client = genai.Client(api_key=api_key)

response = client.models.generate_content(model=model_name, contents="Write a story about a magic backpack.")

print("Response type:", type(response))
print("Response attributes:", dir(response))
print("Response text:", response.text)
print("Full response structure:")
print(json.dumps(response.model_dump(), indent=2))

Response type: <class 'google.genai.types.GenerateContentResponse'>
Response attributes: ['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic

## Tushare

In [6]:
import os

from dotenv import load_dotenv

from utils.my_tushare import convert_to_tushare_code, get_pro_api


load_dotenv()
print(os.getenv("TUSHARE_TOKEN"))

pro = get_pro_api()

ts_code = convert_to_tushare_code("301157")

print(
    pro.bak_daily(
        ts_code=ts_code,
        start_date="20250220",
        end_date="20250221",
        fields="trade_date,open,high,low,close,vol,amount,swing,pct_change,change,turn_over",
    )
)

6637031d14f4036a8dfffe9b186ff41164ddc1d2c9f6c5b521aaa56f
  trade_date   open   high    low  close      vol    amount  swing  \
0   20250221  59.18  66.80  58.01  60.91  94636.0  58995.88  15.74   
1   20250220  54.82  56.18  54.15  55.84  33637.0  18563.50   3.67   

   pct_change  change  turn_over  
0        9.08    5.07      45.81  
1        1.05    0.58      16.28  


## News crawler

In [7]:
from utils.news_crawler import get_stock_news
from utils.my_logging import PROJECT_ROOT
import os
import sys
import json
from datetime import datetime
import time

# 测试参数
test_symbols = ["600519", "300750", "000001"]  # 测试不同的股票代码
test_news_counts = [5, 10, 20]  # 测试不同的新闻数量

print("\n=== 开始测试新闻爬取功能 ===")
print(f"项目根目录: {PROJECT_ROOT}")

# 检查数据目录
news_dir = os.path.join(PROJECT_ROOT, "data", "stock_news")
print(f"新闻存储目录: {news_dir}")

if not os.path.exists(news_dir):
    print(f"创建新闻存储目录: {news_dir}")
    os.makedirs(news_dir, exist_ok=True)

for symbol in test_symbols:
    print(f"\n测试股票代码: {symbol}")
    for news_count in test_news_counts:
        print(f"\n  请求新闻数量: {news_count}")

        try:
            # 爬取新闻
            news_list = get_stock_news(symbol, max_news=news_count)

            # 检查新闻列表
            if news_list:
                print(f"  成功获取新闻数量: {len(news_list)}")

                # 检查新闻文件路径
                news_file = os.path.join(news_dir, f"{symbol}_news.json")

                # 保存新闻数据
                try:
                    save_data = {
                        "date": datetime.now().strftime("%Y-%m-%d"),
                        "news": news_list
                    }
                    with open(news_file, 'w', encoding='utf-8') as f:
                        json.dump(save_data, f,
                                    ensure_ascii=False, indent=2)
                    print(f"  新闻文件已保存: {news_file}")

                    # 验证文件内容
                    with open(news_file, 'r', encoding='utf-8') as f:
                        saved_data = json.load(f)
                        saved_news = saved_data.get("news", [])
                        print(f"  文件中的新闻数量: {len(saved_news)}")
                        print(f"  保存日期: {saved_data.get('date')}")

                        # 检查文件格式
                        if saved_news and isinstance(saved_news[0], dict):
                            print("  新闻格式正确")
                            print(f"  示例新闻标题: {saved_news[0]['title']}")
                        else:
                            print("  警告: 新闻格式可能不正确")
                except Exception as e:
                    print(f"  保存或验证文件时出错: {e}")
            else:
                print("  错误: 未获取到新闻")

        except Exception as e:
            print(f"  测试过程中出错: {e}")

        print("\n  等待5秒后继续下一个测试...")
        time.sleep(5)  # 避免请求过于频繁


2025-03-02 11:34:35,895 [INFO] utils.openrouter_config.<module> (31): ✓ Gemini 客户端初始化成功



=== 开始测试新闻爬取功能 ===
项目根目录: c:\workspace\ai-a-share
新闻存储目录: c:\workspace\ai-a-share\data\stock_news

测试股票代码: 600519

  请求新闻数量: 5
新闻保存目录: c:\workspace\ai-a-share\data\stock_news
成功创建或确认目录存在: c:\workspace\ai-a-share\data\stock_news
新闻文件路径: c:\workspace\ai-a-share\data\stock_news\600519_news.json
开始获取600519的新闻数据...
成功获取到100条新闻
成功添加新闻: 食品饮料行业资金流入榜：贵州茅台、五粮液等净流入资金居前
成功添加新闻: 多家白酒密集停供，茅台批发价迎来上涨
成功添加新闻: 市场成交活跃，融资余额创年内新高，平安中证A50ETF159593)近3月规模、份额增长显著
成功添加新闻: “暂时没有考虑”！贵州茅台副总经理张旭回应茅台提价传言
成功添加新闻: 资本市场情绪积极，市场盈利期望或可进一步放高，平安中证A50ETF159593)近1周新增规模居同类产品第一
成功添加新闻: 出身“茅台酒厂”更受职场欢迎？贵州茅台管理层基本为70后正当年
成功添加新闻: 山科大教授增产茅台30%-50%引争议 茅台基酒产量近10年首降


2025-03-02 11:34:36,101 [INFO] utils.news_crawler.get_stock_news (129): 成功保存5条新闻到文件: c:\workspace\ai-a-share\data\stock_news\600519_news.json


  成功获取新闻数量: 5
  新闻文件已保存: c:\workspace\ai-a-share\data\stock_news\600519_news.json
  文件中的新闻数量: 5
  保存日期: 2025-03-02
  新闻格式正确
  示例新闻标题: 食品饮料行业资金流入榜：贵州茅台、五粮液等净流入资金居前

  等待5秒后继续下一个测试...


2025-03-02 11:34:41,207 [INFO] utils.news_crawler.get_stock_news (129): 成功保存10条新闻到文件: c:\workspace\ai-a-share\data\stock_news\600519_news.json


...


测试股票代码: 000001

  请求新闻数量: 5
新闻保存目录: c:\workspace\ai-a-share\data\stock_news
成功创建或确认目录存在: c:\workspace\ai-a-share\data\stock_news
新闻文件路径: c:\workspace\ai-a-share\data\stock_news\000001_news.json
开始获取000001的新闻数据...
成功获取到100条新闻
成功添加新闻: 国铁集团再与9家全国性股份行签约 推广三款物流金融产品
成功添加新闻: 平安银行同意给予平安租赁综合授信额度等值65亿元
成功添加新闻: 应对大行“下沉战略” 中小银行如何破局
成功添加新闻: 你的信用卡权益“缩水”了吗？信用卡发卡量连降，两年减少约8000万张
成功添加新闻: 方正控股与平安银行签订存款服务协议
成功添加新闻: 平安银行：第十二届董事会第三十五次会议决议公告
成功添加新闻: 平安银行：2024年年度报告计划于2025年3月15日发布
  成功获取新闻数量: 5
  新闻文件已保存: c:\workspace\ai-a-share\data\stock_news\000001_news.json
  文件中的新闻数量: 5
  保存日期: 2025-03-02
  新闻格式正确
  示例新闻标题: 应对大行“下沉战略” 中小银行如何破局

  等待5秒后继续下一个测试...


2025-03-02 11:35:12,023 [INFO] utils.news_crawler.get_stock_news (129): 成功保存10条新闻到文件: c:\workspace\ai-a-share\data\stock_news\000001_news.json



  请求新闻数量: 10
新闻保存目录: c:\workspace\ai-a-share\data\stock_news
成功创建或确认目录存在: c:\workspace\ai-a-share\data\stock_news
新闻文件路径: c:\workspace\ai-a-share\data\stock_news\000001_news.json
缓存的新闻数量(5)不足，需要获取更多新闻(10条)
开始获取000001的新闻数据...
成功获取到100条新闻
成功添加新闻: 国铁集团再与9家全国性股份行签约 推广三款物流金融产品
成功添加新闻: 平安银行同意给予平安租赁综合授信额度等值65亿元
成功添加新闻: 应对大行“下沉战略” 中小银行如何破局
成功添加新闻: 你的信用卡权益“缩水”了吗？信用卡发卡量连降，两年减少约8000万张
成功添加新闻: 方正控股与平安银行签订存款服务协议
成功添加新闻: 平安银行：第十二届董事会第三十五次会议决议公告
成功添加新闻: 平安银行：2024年年度报告计划于2025年3月15日发布
成功添加新闻: 百行千业兴 普惠家国情  平安银行助力民营企业及小微企业发展
成功添加新闻: 鑫磊股份：关于以自有资产抵押向银行申请贷款的公告
成功添加新闻: 大宗交易：机构账户买入平安银行201.44万元（02-17）
成功添加新闻: 平安银行大宗交易成交17.10万股 成交额201.44万元
成功添加新闻: 平安银行：做好股东回报的连续性、稳定性与合理性，充分维护股东依法享有投资收益的权利
成功添加新闻: 银行行业资金流入榜：江苏银行等11股净流入资金超5000万元
成功添加新闻: 解密主力资金出逃股 连续5日净流出525股
成功添加新闻: 银行行业资金流出榜：工商银行、江苏银行等净流出资金居前
  成功获取新闻数量: 10
  新闻文件已保存: c:\workspace\ai-a-share\data\stock_news\000001_news.json
  文件中的新闻数量: 10
  保存日期: 2025-03-02
  新闻格式正确
  示例新闻标题: 应对大行“下沉战略” 中小银行如何破局

  等待5秒后继续下一个测试...


2025-03-02 11:35:17,144 [INFO] utils.news_crawler.get_stock_news (129): 成功保存20条新闻到文件: c:\workspace\ai-a-share\data\stock_news\000001_news.json



  请求新闻数量: 20
新闻保存目录: c:\workspace\ai-a-share\data\stock_news
成功创建或确认目录存在: c:\workspace\ai-a-share\data\stock_news
新闻文件路径: c:\workspace\ai-a-share\data\stock_news\000001_news.json
缓存的新闻数量(10)不足，需要获取更多新闻(20条)
开始获取000001的新闻数据...
成功获取到100条新闻
成功添加新闻: 国铁集团再与9家全国性股份行签约 推广三款物流金融产品
成功添加新闻: 平安银行同意给予平安租赁综合授信额度等值65亿元
成功添加新闻: 应对大行“下沉战略” 中小银行如何破局
成功添加新闻: 你的信用卡权益“缩水”了吗？信用卡发卡量连降，两年减少约8000万张
成功添加新闻: 方正控股与平安银行签订存款服务协议
成功添加新闻: 平安银行：第十二届董事会第三十五次会议决议公告
成功添加新闻: 平安银行：2024年年度报告计划于2025年3月15日发布
成功添加新闻: 百行千业兴 普惠家国情  平安银行助力民营企业及小微企业发展
成功添加新闻: 鑫磊股份：关于以自有资产抵押向银行申请贷款的公告
成功添加新闻: 大宗交易：机构账户买入平安银行201.44万元（02-17）
成功添加新闻: 平安银行大宗交易成交17.10万股 成交额201.44万元
成功添加新闻: 平安银行：做好股东回报的连续性、稳定性与合理性，充分维护股东依法享有投资收益的权利
成功添加新闻: 银行行业资金流入榜：江苏银行等11股净流入资金超5000万元
成功添加新闻: 解密主力资金出逃股 连续5日净流出525股
成功添加新闻: 银行行业资金流出榜：工商银行、江苏银行等净流出资金居前
成功添加新闻: 债市春寒 全网都在劝我“固收+”
成功添加新闻: 金价创新高后“紧箍咒”来袭 中行、平安上调黄金现货延期合约保证金比例
成功添加新闻: 银行发布短视频人才“招募令”，争夺流量任务有多重？
成功添加新闻: 3.29亿元主力资金今日撤离银行板块
成功添加新闻: 银行行业今日涨1.05%，主力资金净流入7.99亿元
成功添加新闻: 逼近900元/克！黄金怎么买更合适？
成功添加新闻: 黄金火热！在银行和金店买有什么区别？怎么买更